In [3]:
# import the genai types module (overrides the built-in 'types' if already imported)
import google.genai.types as types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,  # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [1]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, AgentTool, FunctionTool

print("✅ ADK components imported successfully.")

c:\Users\Keskinen\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


✅ ADK components imported successfully.


Define multi-agent system like having a team of specialists, each one focus on ONE clear task.

Research Agent - Searches for information using Google Search; 
Summarizer Agent - Creates concise summaries from research findings


In [9]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="Research_Agent",
    description="An agent that searches for information using Google Search.",
    model=Gemini(model_name="gemini-2.0-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized researcg agent. Your only job is to use the google_search tool to find 2-3 pieces of relevant information on the given topic. And present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ Research Agent defined successfully.")

# Summarizer Agent: Its job is to take the research findings from research_agent and create a concise summary.
summarizer_agent = Agent(
    name="Summarizer_Agent",
    description="An agent that summarizes research findings.",
    model=Gemini(model_name="gemini-2.0-flash-lite", retry_options=retry_config),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",  # The result of this agent will be stored in the session state with this key.
)

print("✅ Summarizer Agent defined successfully.")

✅ Research Agent defined successfully.
✅ Summarizer Agent defined successfully.


In [11]:
# Root Coordinator/Root Agent: Orchestrates the workflow by calling the sub-agents as tools.

root_agent = Agent(
    name="Root_Coordinator_Agent",
    description="An agent that orchestrates the research and summarization process.",
    model=Gemini(model_name="gemini-2.0-flash-lite", retry_options=retry_config),
    # This instruction tells the root agent HOW to use its tools (the sub-agents).
    instruction="""You are the root coordinator agent. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `Research_Agent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `Summarizer_Agent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ Root Coordinator Agent defined successfully.")

✅ Root Coordinator Agent defined successfully.


In [12]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Search for job postings for AI Product Designer and AI product manager roles in the EU in November 2025 and summarize the key skills and qualifications required for these positions."
)


 ### Created new session: debug_session_id

User > Search for job postings for AI Product Designer and AI product manager roles in the EU in November 2025 and summarize the key skills and qualifications required for these positions.


Root_Coordinator_Agent > Here's a summary of the key skills and qualifications required for AI Product Designer and AI Product Manager roles in the EU, based on job postings from November 2025:

*   **Deep Understanding of AI/ML/GenAI:** Both roles critically require a strong understanding of AI/ML fundamentals, a deep interest in artificial intelligence, and experience in developing, steering, or designing for AI-powered products and platforms.
*   **Extensive Product Experience:** Candidates for both positions are typically expected to have significant experience (often 5+ years) in their respective fields: product management (preferably B2B/B2C SaaS) or end-to-end product design, specifically for technology products.
*   **Focus on AI Product Strategy & User Experience:** AI Product Managers drive AI product vision, strategy, and development, including GenAI-powered products. AI Product Designers specialize in crafting intuitive, scalable, and innovative user interfaces and experien